🤖 **[View in NotebookLM Research Assistant](https://notebooklm.cloud.google.com/us/notebook/412eb240-633b-47a6-b963-7c9f50409f70?project=294334118581)**

_This notebook is automatically synced to NotebookLM daily for AI-powered research assistance._

In [1]:
from IPython.display import Markdown
import pandas as pd
from tulip.core.collection import TulipCollection
from tulip.data.bloomberg import BloombergClient as bb
from tulip.data.haver import HaverClient as hc
from tulip.plots import plot_area, plot_lines

from tulip.data.gs import GSClient as gs

import pycountry

from tulip.analysis.country_related.analytics import summarize_gs_eco_fct

ISO_2 = "CA"

pyctry = pycountry.countries.get(alpha_2=ISO_2)
import plotly.offline as pyo

pyo.init_notebook_mode(connected=True)

Haver path setting remains unchanged.



## Canada
### Activity Indicators
#### Economic Forecasts (Brokers)

In [2]:
gs_eco_fct = gs.get_eco_forecast(geographyId=ISO_2)
gs_summary = summarize_gs_eco_fct(gs_eco_fct)
gs_summary = gs_summary[~gs_summary.index.str.contains("ngdp")].to_frame().T
gs_summary.style.set_caption(f"Goldman {pyctry.name} Economic Forecasts").format(
    precision=2
)

metric,core_cpi,cpi_avg,current_account,rgdp_qoq,rgdp_yoy,output_gap
forecastValue,3.00,2.03,-2.29,1.20,1.18,-2.21


#### Current Activity Indicator (Goldman)

In [3]:
cai_series_soft_vs_hard = gs.get_CAI_series(
    geographyId=ISO_2,
    metricName=[
        "CAI_HEADLINE",
        "CAI_CONTRIBUTION_TYPE_HARD",
        "CAI_CONTRIBUTION_TYPE_SOFT",
    ],
    startDate="1980-01-01",
)
cai_series_soft_vs_hard = cai_series_soft_vs_hard.set_index("metricName", append=True)[
    "metricValue"
].unstack("metricName")
cai_series_soft_vs_hard.columns = ["Hard", "Soft", "Headline"]
cai_plot = plot_lines(
    cai_series_soft_vs_hard,
    show_0=True,
    title=f"<b>{pyctry.name} Current Activity Indicator</b> Updated: {pd.Timestamp.today().strftime('%Y-%m-%d')}",
    years_limit=4,
)
cai_plot

#### IFSD Nowcast
[IFSD](https://ifsd.ca/) was founded in 2016 by Kevin Page, Canada’s first Parliamentary Budget Officer. IFSD is both a non-profit, Canadian think-tank and a global advisory practice

In [4]:
ifs_nowcast = bb.get_series("IFSDCAST Index")
ifs_nowcast.plot(years_limit=2, tick_suffix="%")

#### Retail Sales

In [5]:
retail_sales = bb.get_series("CARSCHNG Index")
retail_sales.info.title = "Retail Sales MoM"
retail_sales.plot(years_limit=2, tick_suffix="%")

#### PMIs

In [6]:
pmi = {
    "MPMICASA Index": "S&P Manufacturing PMI",
    "MPMICACA Index": "S&P Composite PMI",
    "IVEYSA   Index": "Ivey PMI",
}

pmi_collection = bb.create_collection(list(pmi.keys()))
for i, k in enumerate(pmi.values()):
    pmi_collection[i].info.title = k

pmi_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Quote Units,Updated
S&P Manufacturing PMI,50.5,2025-10-31,46.3,4.2,2.1,9.0,0.10,-,2025-11-12 19:17:00
S&P Composite PMI,50.3,2025-10-31,46.3,4.0,2.1,8.6,-0.40,-,2025-11-12 19:17:00
Ivey PMI,52.4,2025-10-31,59.8,-7.4,-1.1,4.5,0.40,-,2025-11-12 19:17:00


In [7]:
pmi_collection.dashboard.plots(hlines=50)

#### Growth Stats
##### Real GDP Growth

In [8]:
real_gdp_yoy = hc.get_series("V6E05783@CANADA")
real_gdp_yoy.ts = real_gdp_yoy.ts.pct_change(4)
real_gdp_yoy.plot(show_0=True, tick_format="0.0%")

##### GDP Detail

In [9]:
growth_stats = {
    "Real GDP": "yryr%(V6E05752@CANADA)",  # Canada: Real Gross Domestic Product at Market Prices
    "Fixed Investment": "yryr%(V6E05732@CANADA)",  # Canada: GDP: Gross Fixed Capital Formation
    "Inventories": "V6E05741@CANADA",  # Canada: GDP: Investment in Inventories
    "Consumption": "yryr%(V6E05723@CANADA)",  # Canada: GDP: Final Consumption Expenditure
    "Exports": "yryr%(V6E05745@CANADA)",  # Canada: GDP: Exports of Goods and Services
    "Imports": "yryr%(V6E05748@CANADA)",  # Canada: GDP: Imports of Goods and Services
}

growth_stats_collection = []
for k, v in growth_stats.items():
    growth_stats_collection.append(hc.get_series(v))

growth_stats_collection = TulipCollection(growth_stats_collection)
growth_stats_collection["yryr%(V6E05748@CANADA)"].good_is = -1
growth_stats_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Canada: Gross Domestic Product at Market Prices (SAAR, Mil.Chn.2017.C$)",1.2,2025-06-30,2.3,-1.0,-0.61,-0.73,0.01,2025-11-12 19:17:00
"Canada: GDP: Gross Fixed Capital Formation: Total (SAAR, Mil.Chn.2017.C$)",0.34,2025-06-30,1.3,-0.91,-0.24,1.2,1.4,2025-11-12 19:17:00
"Canada: GDP: Investment in Inventories: Total (SAAR, Mil.Chn.2017.C$)","26,898",2025-06-30,"7,913","18,985",2.3,"6,865","1,621",2025-11-12 19:17:00
"Canada: GDP: Final Consumption Expenditure (SAAR, Mil.Chn.2017.C$)",3.4,2025-06-30,2.9,0.46,0.29,0.49,1.1,2025-11-12 19:17:00
"Canada: GDP: Exports of Goods and Services: Total (SAAR, Mil.Chn.2017.C$)",-4.7,2025-06-30,1.8,-6.5,-1.5,-5.4,-4.6,2025-11-12 19:17:00
"Canada: GDP: Imports of Goods and Services: Total (SAAR, Mil.Chn.2017.C$)",-0.07,2025-06-30,1.3,-1.3,-0.25,-0.06,-0.64,2025-11-12 19:17:00


In [10]:
growth_stats_collection.dashboard.plots(show_0=True)

##### Nominal GDP

In [11]:
#  Nominal GDP Growth
hc.get_series("yryr%(V6E05783@CANADA)").plot(show_0=True)  # Nominal GDP Growth

### Prices

In [12]:
prices = {
    "CPI-Trim": "V1M85715@CANADA",
    "CPI-Median": "V1M85714@CANADA",
    "CPI-Common": "V1M85713@CANADA",
    "CPI YoY": "n156cy@g10",
    "Wages YoY": "canewcl@CANADA",
    "Inflation Expectations 1Y": "CANVXI1@CANADA",  # CSCE
    "Wage Growth Expectations 1Y": "CANVXW1@CANADA",
}

prices_collection = []
for k, v in prices.items():
    prices_collection.append(hc.get_series(v))

prices_collection = TulipCollection(prices_collection)
prices_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Canada: Bank of Canada: Core Inflation: CPI-Trim (SA, Yr/Yr %Chg)",3.1,2025-09-30,3.0,0.10,0.67,0.20,0.60,2025-11-12 19:17:00
"Canada: Bank of Canada: Core Inflation: CPI-Median (SA, Yr/Yr %Chg)",3.2,2025-09-30,3.2,0.00,0.00,0.30,0.90,2025-11-12 19:17:00
"Canada: Bank of Canada: Core Inflation: CPI-Common (NSA, Yr/Yr %Chg)",2.7,2025-09-30,2.5,0.20,1.2,0.20,0.60,2025-11-12 19:17:00
"Canada: Consumer Price Index (NSA, Y/Y, %Change)",2.4,2025-09-30,1.9,0.50,0.69,-0.29,0.72,2025-11-12 19:17:00
Canada: Bank of Canada Wage-Common: Hourly Wage from LFS (Y/Y % Change),3.3,2025-09-30,3.4,-0.10,-0.10,-1.1,-1.6,2025-11-12 19:17:00
Canada: Consumer Expectations Survey: Inflation 1 Year Ahead (%),4.0,2025-09-30,4.0,-0.04,-0.07,0.95,0.16,2025-11-12 19:17:00
Canada: Consumer Expectations Survey: Wage Growth Over Next 12 Months (%),2.7,2025-09-30,2.5,0.12,0.89,0.01,0.01,2025-11-12 19:17:00


In [13]:
prices_df = prices_collection.df
prices_df.columns = list(prices.keys())
fig = plot_lines(
    prices_df.iloc[:, :3],
    years_limit=5,
    title="<b>Core Inflation Measures</b> (Preferred by the BoC)",
    tick_suffix="%",
)
fig.add_hline(2, line_width=1, annotation_text="Target")  # , annotation="Target")

In [14]:
fig = plot_lines(
    prices_df.iloc[:, -3:].ffill(),
    years_limit=5,
    title="<b>Inflation Surveys</b>",
    tick_suffix="%",
)
fig.add_hline(2, line_width=1, annotation_text="Target")  # , annotation="Target")

In [15]:
business_outlook_survey_prices_dict = {
    "Below 1 percent": "B156IE1@CANADA",
    "1 to 2 percent": "B156IE2@CANADA",
    "2 to 3 percent": "B156IE3@CANADA",
    "Above 3 percent": "B156IE4@CANADA",
    "No response": "B156IE5@CANADA",
}

business_outlook_survey_prices = []
for k, v in business_outlook_survey_prices_dict.items():
    business_outlook_survey_prices.append(hc.get_series(v))

business_outlook_survey_prices = TulipCollection(business_outlook_survey_prices)
business_outlook_survey_prices.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Canada: Business Outlook: Inflation Expectations: Below 1 Percent (% of Firms),0.00,2025-09-30,1.0,-1.0,-0.18,0.00,-2.0,2025-11-12 19:17:00
Canada: Business Outlook: Inflation Expectations: 1 to 2 Percent (% of Firms),21.0,2025-09-30,12.0,9.0,0.81,-3.0,11.0,2025-11-12 19:17:00
Canada: Business Outlook: Inflation Expectations: 2 to 3 Percent (% of Firms),51.0,2025-09-30,51.0,0.00,0.00,0.00,-11.0,2025-11-12 19:17:00
Canada: Business Outlook: Inflation Expectations: Above 3 Percent (% of Firms),18.0,2025-09-30,23.0,-5.0,-0.63,-2.0,3.0,2025-11-12 19:18:00
Canada: Business Outlook: Inflation Expectations: No Response (% of Firms),10.0,2025-09-30,14.0,-4.0,-1.4,5.0,-1.0,2025-11-12 19:18:00


In [16]:
bos_prices = business_outlook_survey_prices.df
bos_prices.columns = list(business_outlook_survey_prices_dict.keys())

# Define custom colors for each category
colors = {
    "Above 3 percent": "red",
    "2 to 3 percent": "coral",
    "1 to 2 percent": "lightgreen",
    "Below 1 percent": "green",
    "No response": "gray",
}

fig = plot_area(
    bos_prices,
    title="<b>Business Outlook Survey - Inflation Expectations</b>",
    tick_suffix="%",
    logo=False,
)

for trace in fig.data:
    if trace.name in colors:
        trace.line.color = colors[trace.name]

fig.show()

Other faster, idiosyncratic price data points available in Canada

In [17]:
weekly_price = {
    "Commodity Price Index": "P156CP@INTWKLY",
    "Commodity Price Index Ex Energy": "P156CPX@INTWKLY",
    "Gasoline Price": "P156GRT@INTWKLY",
    "Pork Prices": "P156HGV@INTWKLY",
}

weekly_price_collection = []
for k, v in weekly_price.items():
    weekly_price_collection.append(hc.get_series(v))

weekly_price_collection = TulipCollection(weekly_price_collection)
weekly_price_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Canada: Commodity Price Index (Jan-72=100),618.8,2025-11-05,612.6,6.2,0.67,19.4,24.4,2025-11-12 19:18:00
Canada: Commodity Price Index: Excluding Energy (Jan-72=100),529.3,2025-11-05,537.1,-7.8,-2.2,20.7,66.4,2025-11-12 19:18:00
"Canada: Volume Weighted Average Regular Gasoline Price (EOP, CAD Cents/liter)",142.0,2025-11-11,141.4,0.60,0.23,0.60,-10.6,2025-11-12 19:18:00
Ontario Pork Prices: Avg Price (CAD/ckg Dressed Weight),272.4,2025-10-31,277.7,-5.2,-0.75,19.1,22.9,2025-11-12 19:18:00


### Employment

In [18]:
# Employment metrics
employment = {
    "EI Claims": "E156CEA@INTWKLY",  # Canada: Employment Insurance: Claims Established
    "EI Recipients": "E156ATA@INTWKLY",  # Canada: Employment Insurance: Active Recipients
    "Job Loss Probability": "CANVXJL@CANADA",  # Canada: Consumer Expectations Survey: Prob of Losing Job
}

employment_collection = []
for k, v in employment.items():
    employment_collection.append(hc.get_series(v))

employment_collection = TulipCollection(employment_collection)
employment_collection.good_is = {
    "E156CEA@INTWKLY": -1,  # Lower claims is good
    "E156ATA@INTWKLY": -1,  # Lower recipients is good
    "CANVXJL@CANADA": -1,  # Lower probability of job loss is good
}
employment_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Canada: Employment Insurance: Claims Established [Approved Apps] (Persons),"47,850",2025-10-27,"43,880","3,970",0.34,"10,600","4,770",2025-11-12 19:18:00
Canada: Employment Insurance: Active Recipients (Persons),"924,740",2025-10-27,"904,760","19,980",0.70,"-169,380","77,470",2025-11-12 19:18:00
Canada: Consumer Expectations Survey: Prob of Losing Job Over Next 12 Mo (%),17.4,2025-09-30,19.6,-2.2,-0.96,2.1,6.0,2025-11-12 19:18:00


In [19]:
employment_collection.dashboard.plots()

### Output Gap

In [20]:
output_gap = hc.get_series("B156IGAP@CANADA")
output_gap.plot(show_0=True, tick_suffix="%")

In [21]:
weekly_price_collection.dashboard.plots(years_limit=3)

### Trade

In [22]:
trade_stats = {
    "Current Account Balance % GDP": "s156bcpg@g10",
}

trade_collection = []
for k, v in trade_stats.items():
    trade_collection.append(hc.get_series(v))

trade_collection = TulipCollection(trade_collection)
trade_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Canada: Current Account Balance as a Percentage of GDP (SA, %)",-2.7,2025-06-30,-0.17,-2.5,-2.9,-2.3,-2.0,2025-11-12 19:18:00


In [23]:
trade_collection.dashboard.plots(show_0=True)

### Surveys

In [24]:
surveys = {
    "Business Barometer 3M": "CFO3@CANADA",  # Canada: Business Barometer Index: 3 Month Outlook
    "Business Barometer 1Y": "CFAA@CANADA",  # Business Barometer Index: Canada
    "Policy Uncertainty": "CEPUIN@CANADA",  # Canada: News-Based Policy Uncertainty Index
}

surveys_collection = []
for k, v in surveys.items():
    surveys_collection.append(hc.get_series(v))

surveys_collection = TulipCollection(surveys_collection)
surveys_collection.good_is = {
    "CFO3@CANADA": 1,  # Higher business confidence is good
    "CFAA@CANADA": 1,  # Higher business confidence is good
    "CEPUIN@CANADA": -1,  # Lower uncertainty is good
}
surveys_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Canada: Business Barometer Index: 3 Month Outlook (NSA, >50=Stronger)",44.8,2025-10-31,44.6,0.20,0.04,4.3,-3.5,2025-11-12 19:18:00
"Business Barometer Index: Canada (NSA, >50=Stronger in the Next Year)",46.3,2025-10-31,50.3,-4.0,-0.91,11.6,-9.5,2025-11-12 19:18:00
Canada: News-Based Policy Uncertainty Index (Mean=100 Prior to 2011),688.3,2025-08-31,"1,057",-368.2,-5.0,-614.7,393.2,2025-11-12 19:18:00


In [25]:
surveys_collection.dashboard.plots(years_limit=3, hlines=[50, 50])

### Credit Creation

In [26]:
debt_stats = {
    "Non-Financial Debt % GDP": "S156zdnp@g10",
    "Household Debt to Disposable Income": "S156zdhi@g10",
    "Household Debt % GDP": "S156zdhp@g10",
}

debt_collection = []
for k, v in debt_stats.items():
    debt_collection.append(hc.get_series(v))

debt_collection = TulipCollection(debt_collection)
debt_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Canada: Nonfin Private Corporations Debt Outstanding as a % of SA GDP(%),74.9,2025-06-30,74.3,0.59,0.30,1.4,2.2,2025-11-12 19:18:00
Canada: HHs & Nonprofit Institutions: Debt as a % of SA Disposable Income(%),162.7,2025-06-30,160.8,1.9,0.87,1.8,-0.06,2025-11-12 19:18:00
Canada: HHs & Nonprofit Institutions: Debt as a % of SA GDP(%),99.1,2025-06-30,97.3,1.9,0.98,0.92,1.0,2025-11-12 19:18:00


In [27]:
debt_collection.dashboard.plots()

### Budget

In [28]:
government_finance = {
    "Public Debt": "N156FDP@G10",  # Canada: Business Barometer Index: 3 Month Outlook
    "Govt Budget Surplus/Deficit": "H156FGBP@G10",  # Business Barometer Index: Canada
}

government_finance_collection = []
for k, v in government_finance.items():
    government_finance_collection.append(hc.get_series(v))

government_finance_collection = TulipCollection(government_finance_collection)

government_finance_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Canada: General Government Debt as a Percentage of Annualized SWDA GDP (%),92.3,2025-06-30,90.7,1.6,0.43,2.9,5.8,2025-11-12 19:18:00
"Canada: Federal Government Budget Surplus/Deficit as a % of SA GDP(SA, %)",-1.9,2025-06-30,-1.5,-0.41,-0.18,-0.73,0.07,2025-11-12 19:18:00


In [29]:
government_finance_collection.dashboard.plots()

### Asset Pricing

In [30]:
# Asset Piricing

# Yields and Stock

In [31]:
Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")

_Notebook updated at 2025-11-12 19:18_